## Create Synthetic Dates

Features of the adjusted data:
Specific categories: Using only the categories present in your data

Realistic distribution: Based on observed frequency in the provided data

Realistic parameters per category:

🥦 **Produce**
- **Lead Time:** 1–3 days (locally sourced), 5–10 days (imported)
- **Shelf Life:** 3–10 days (most fresh items), up to 2 weeks for hardy vegetables like carrots or potatoes

🌾 **Grains and Flours**
- **Lead Time:** 3–7 days (domestic), 10–15 days (imported specialty grains)
- **Shelf Life:** 6 months to 1 year (dry, sealed), up to 2 years for rice and flour stored properly

🧀 **Dairy and Cold Cuts**
- **Lead Time:** 2–5 days (regional suppliers), 7–10 days (specialty cheeses)
- **Shelf Life:**
  - Milk & cream: 7–14 days refrigerated
  - Yogurt & soft cheeses: 2–3 weeks
  - Hard cheeses: 1–3 months
  - Cold cuts: 1–2 weeks sealed

☕ **Beverages**
- **Lead Time:** 2–7 days (coffee/tea distributors)
- **Shelf Life:**
  - Tea: 1–2 years (dry)
  - Coffee beans: 6–12 months (sealed), 1–2 weeks after grinding
  - Brewed drinks: 1–3 days refrigerated

🥚 **Eggs and Poultry**
- **Lead Time:** 1–3 days (local farms), 5–7 days (wholesale)
- **Shelf Life:**
  - Eggs: 3–5 weeks refrigerated
  - Fresh poultry: 1–2 days raw, 3–4 days cooked

🐟 **Meats and Fish**
- **Lead Time:** 1–5 days (fresh), 7–10 days (frozen or imported)
- **Shelf Life:**
  - Fresh fish: 1–2 days
  - Frozen fish: 3–6 months
  - Cured fish (e.g., sardines): up to 1 year

🛢️ **Oils and Fats**
- **Lead Time:** 3–7 days (bulk suppliers)
- **Shelf Life:**
  - Vegetable oils: 6–12 months
  - Butter: 1 month refrigerated, 6 months frozen
  - Coconut oil: up to 2 years

🍬 **Sugars and Sweets**
- **Lead Time:** 2–5 days
- **Shelf Life:**
  - Sugars: indefinite if dry and sealed
  - Dried fruits (e.g., plum): 6–12 months

🍪 **Miscellaneous and Biscuits**
- **Lead Time:** 2–6 days
- **Shelf Life:**
  - Biscuits: 3–6 months sealed


Seasonal patterns:

- Fruits/vegetables with reduced shelf life in summer

- Dairy with shorter lead time in winter

Realistic temporal distribution:

- 80% of deliveries on weekdays

Controlled outliers: Only 3% of data with unusual situations

These synthetic data preserve the specific characteristics of the categories in your original dataset, with realistic temporal relationships for supply chain analysis.

## Data Generation
### Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import json

from smart_supply_chain_ai.utils import create_data_functions, combine_df_functions

import warnings
warnings.filterwarnings('ignore')

### Paths

In [ ]:
# Define data paths
raw_data_path = os.path.join('../data', 'raw/')

external_data_path = os.path.join('../data', 'external/')

json_path = os.path.join('../src','smart_supply_chain_ai' , 'utils/')

In [ ]:
# List of JSON filenames (without extension) to be loaded
arch_json = ['products','products_categories', 'suppliers']

# Dictionary to store the loaded JSON content
store_catalog = {}

# Loop through each filename, build the full path, and load the JSON data
for name in arch_json:
    file_path = os.path.join(json_path, f"{name}.json")  # Construct full file path
    with open(file_path, "r", encoding="utf-8") as f:     # Open the JSON file
        store_catalog[name] = json.load(f)                        # Load and store the data under its name

# Product catalog information

In [ ]:
# Create a DataFrame of products with product names as a column
products = pd.DataFrame.from_dict(store_catalog['products']).T.reset_index().rename(columns={'index': 'product'})


In [ ]:
# Replace product with new IDs
products['product_id'] = create_data_functions.create_IDs(products.shape[0], suffix='P')

# Supplier catalog and distribution details

In [ ]:
# Create a DataFrame of suppliers with supplier names as a column
suppliers = pd.DataFrame.from_dict(store_catalog['suppliers']).T.reset_index().rename(columns={'index': 'supplier'})

In [ ]:
# Insert supplier IDs as the second column
suppliers.insert(1, 'supplier_id', create_data_functions.create_IDs(suppliers.shape[0], suffix='S'))

In [ ]:
# Remove 'category' and 'subcategories' columns from the suppliers DataFrame
suppliers.drop(columns=['category', 'subcategories'], inplace=True)


In [ ]:
# Split each supplier's product list into separate rows and reset the index
suppliers = suppliers.explode('products').reset_index(drop=True)


In [ ]:
# Merge product and supplier data on matching product names, then drop duplicate 'products' column from suppliers
supply_df = pd.merge(products, suppliers, left_on='product', right_on='products').drop(columns='products')


In [ ]:
# Initialize a random number generator with a fixed seed for reproducibility.
rng = np.random.default_rng(seed=43)
# Assign random supplier ratings between 1 and 4 to all suppliers.
supply_df['supplier_rating'] = rng.integers(1, 5, size=supply_df.shape[0])
# Randomly select 15 unique suppliers to be considered "top suppliers".
suppliers_top = np.random.choice(supply_df['supplier'].unique(), 15, replace=False)
# Update ratings: if the supplier is in the top list, set rating to 5; otherwise keep the original rating.
supply_df['supplier_rating'] = np.where(supply_df['supplier'].isin(suppliers_top), 5, supply_df['supplier_rating'])

## Meteorological Data for Supply Chain Management

In [ ]:
# Set the path to the weather CSV file
archive_csv = external_data_path + 'dados_83967_D_2015-01-01_2025-09-18.csv'

# Read the CSV file into a DataFrame
weather_df = pd.read_csv(archive_csv, sep=";", decimal=",", skiprows=9, engine="python")

# Show the first rows of the DataFrame
weather_df.head()

In [ ]:
# Remove columns that contain only missing values
weather_df.dropna(axis=1, how='all', inplace=True)


In [ ]:
# Rename columns to clear and descriptive English names
weather_df.columns = [
    "measurement_date",
    "daily_total_precipitation_mm",
    "daily_maximum_temperature_c",
    "daily_minimum_temperature_c",
    "daily_average_wind_speed_mps"
]


In [ ]:
# Set 'measurement_date' as index and remove rows with all missing values
weather_df.set_index('measurement_date').dropna(how='all', inplace=True)

# Show the first rows of the DataFrame
weather_df.head()


In [ ]:
# Convert 'measurement_date' column to datetime format
weather_df['measurement_date'] = pd.to_datetime(weather_df['measurement_date'])


In [ ]:
# Display summary information about the DataFrame
weather_df.info()

In [ ]:
# Dropped all missing values
weather_df.dropna(inplace=True)

In [ ]:
# Apply the weather classification function to the cleaned DataFrame to generate severity and category labels
weather_severity_df = create_data_functions.classify_weather(weather_df)

In [ ]:
# Show 10 samples rows of the DataFrame
weather_severity_df.sample(10)

In [ ]:
# Generate and transpose summary statistics for all numeric columns in the classified weather DataFrame
weather_severity_df.describe().T

In [ ]:
# Define the start date of the time range
start_date = pd.to_datetime('2020-01-01')

# Define the end date
end_date = pd.to_datetime('2025-01-05')

# Create a daily date range from start_date to end_date
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Specify the number of rows in the dataset (i.e., total number of records to generate)
n_rows = 6000

In [ ]:
# Randomly sample dates from the date range
random_dates = np.random.choice(date_range, size=n_rows, replace=True)

# Create a DataFrame with the sampled dates
date_df = pd.DataFrame({
    'LPO': random_dates
})

In [ ]:
# Apply the day classification function to each value in the 'LPO' (latest_purchase_order) column and store the result in a new column
date_df['day_classification'] = date_df['LPO'].apply(create_data_functions.day_classification)

In [ ]:
# Merge weather severity data into the date DataFrame based on matching dates ('LPO' (latest_purchase_order) and 'measurement_date'),
# then drop the redundant 'measurement_date' column after the join
climate_date_df = pd.merge(date_df, weather_severity_df, left_on='LPO', right_on='measurement_date', how='inner').drop(columns='measurement_date')


# Realistic supply chain modeling based on weather and product data

In [ ]:
# Create a copy of the climate data DataFrame to work with weather-specific analysis
df_weather = climate_date_df.copy()

# Create a copy of the supply data DataFrame to work with product-related operations
df_products = supply_df.copy()

In [ ]:
# Display the first row of the weather DataFrame to preview its structure
df_weather.head(1)


In [ ]:
# Preview the first row of the products DataFrame to check column names and initial data
df_products.head(1)

In [ ]:
# Converter a coluna 'LPO' para o formato datetime
df_weather['LPO'] = pd.to_datetime(df_weather['LPO'])

# Extrair o número do mês da data
df_weather['month'] = df_weather['LPO'].dt.month

# Extrair o nome do mês (ex: Janeiro, Fevereiro)
df_weather['month_name'] = df_weather['LPO'].dt.month_name()

# Extrair o nome do dia da semana (ex: Segunda-feira)
df_weather['day_of_week'] = df_weather['LPO'].dt.day_name()

# Extrair o número do dia do mês
df_weather['day_of_month'] = df_weather['LPO'].dt.day

# Criar uma coluna booleana indicando se o dia é feriado
df_weather['is_holiday'] = np.where(df_weather['day_classification'] == 'Holiday', True, False)

# Criar uma coluna booleana indicando se o dia é fim de semana (sábado ou domingo)
df_weather['is_weekend'] = np.where(df_weather['LPO'].dt.dayofweek > 4, True, False)


In [ ]:
# Display the first few rows of the weather DataFrame
df_weather

In [ ]:
# Display the first few rows of the products DataFrame
df_products

In [ ]:
# Initialize the supply chain simulator with weather and product data,
# then run the simulation to generate a combined DataFrame with results.
simulator = combine_df_functions.SupplyChainSimulator(df_weather, df_products)
df_combined = simulator.run_simulation()


In [ ]:
# Display DataFrame
df_combined

In [ ]:
# Create a delivery plan with up to 5 products per day.
# Allows 30% of products to be out-of-season.
deliveries, stats = simulator.create_balanced_delivery(max_products_per_day=5, out_of_season_percentage=0.3)

In [ ]:
# Display DataFrame
stats

In [ ]:
# Display DataFrame
deliveries

In [ ]:
# Save Data
deliveries.to_csv(raw_data_path + 'grocery_data.csv', index=False)
stats.to_csv(raw_data_path + 'grocery_stats.csv', index=False)